<a href="https://colab.research.google.com/github/DenisDrobyshev/university/blob/master/whisper_inference_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5ycnxrax
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5ycnxrax
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 38.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [42]:
import whisper
import torch
from pytorch_lightning import LightningModule
import IPython
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [43]:
class Config:
    warmup_steps = 2
    batch_size = 16
    num_worker = 2
cfg = Config()
class Base_rus(LightningModule):
    def __init__(self, cfg:Config, model_name="base", lang="ru") -> None:
        super().__init__()
        self.options = whisper.DecodingOptions(language=lang, without_timestamps=True)
        self.model = whisper.load_model(model_name)
        self.tokenizer = whisper.tokenizer.get_tokenizer(True, language="ru", task=self.options.task)
        self.cfg = cfg
    def forward(self, x):
        return self.model(x)
base_rus_model = Base_rus(cfg)
woptions = whisper.DecodingOptions(language="ru", without_timestamps=True)

In [44]:
name='/content/sample_data/data_whisper/original (1).wav'
IPython.display.Audio(name)

In [46]:
audio = whisper.load_audio(name)
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(DEVICE)

In [47]:
result = base_rus_model.model.decode( mel,woptions)
print(result.text)

Но глазам и бегали. А скажите, вы не манархист? Читал машинка я ваш роман в журнале страна. Дымчатый точь изверь был заинтересован в том, чтобы ничего не случилось. В один вечер я прочитал примерно четверть моего романа. Я зажок керосинку напалу в гу.


In [ ]:
# ЗАПУСКАТЬ ЭТОТ ФАЙЛ ЕСЛИ НЕТ В КОЛЛАБЕ ФАЙЛА.pt (весов модели)
import os.path
if not os.path.isfile('base_ru.pt'):
    import requests
    from urllib.parse import urlencode

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/d/ogeKhA-PfNpf9w' # ваша ссылка только

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    with open('base_ru.pt', 'wb') as f:   # Здесь укажите нужный путь к файлу
        f.write(download_response.content)

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_rus_model.load_state_dict(
    torch.load('/content/sample_data/data_whisper/base_ru.pt', map_location=device)
)
base_rus_model.eval()
base_rus_model = base_rus_model.to(device)

In [49]:
result = base_rus_model.model.decode( mel,woptions)
print(result.text)

двадцать два тюрьма и бедли а скажите вы не мы на архитал ли насчика я ваширом в журнале страна
